# ch3

## 3.1

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [ ]:
html = urlopen("https://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html.read(), "lxml")

In [ ]:
for link in bsObj.findAll("a"):
    if "href" in link.attrs:
        print(link.attrs["href"])

In [ ]:
for link in bsObj.find("div", {"id": "bodyContent"}).findAll("a", href = re.compile("^(/wiki/)((?!:).)*$")):
    if "href" in link.attrs:
        print(link.attrs["href"])

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

In [ ]:
random.seed(datetime.datetime.now())

In [ ]:
def getLinks(articleUrl):
    html = urlopen("https://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html, "lxml")
    return bsObj.find("div", {"id": "bodyContent"}).findAll("a", href = re.compile("^(/wiki/)((?!:).)*$"))

In [ ]:
links = getLinks("/wiki/Kevin_Bacon")

In [ ]:
while len(links)>0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

## 3.2 採集整個網站

### 3.2 ex1

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [ ]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("https://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "lxml")
    for link in bsObj.findAll("a", href = re.compile("^(/wiki/)")):
        if("href" in link.attrs):
            if(link.attrs["href"] not in pages):
                #Face new page
                newPage = link.attrs["href"]
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

In [ ]:
getLinks("")

### 3.2 ex2

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [ ]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("https://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "lxml")
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id = "mw-content-text").findAll("p")[0])
        print(bsObj.find(id = "ca-edit").find("span").find("a").attrs["href"])
    except AttributeError:
        print("頁面缺少一些屬性，不過不用擔心")
    for link in bsObj.findAll("a", href = re.compile("^(/wiki/)")):
        if("href" in link.attrs):
            if(link.attrs["href"] not in pages):
                #Face new page
                newPage = link.attrs["href"]
                print("--------------------\n"+newPage)
                pages.add(newPage)
                getLinks(newPage)

In [ ]:
getLinks("")

3.3 通過互聯網採集

ex1

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

In [ ]:
pages = set()
random.seed(datetime.datetime.now())

In [ ]:
def getInternalLinks(bsObj, includeUrl):
    #獲取頁面所有的內鏈列表
    internalLinks = []
    #找出所有以"/"開頭的鏈結
    for link in bsObj.findAll("a", href = re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs["href"] is not None:
            if( link.attrs["href"] not in internalLinks):
                internalLinks.append(link.attrs["href"])
    return internalLinks

In [ ]:
startingPage = "http://oreilly.com"
html = urlopen(startingPage)
bsObj = BeautifulSoup(html, "lxml")

In [ ]:
def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    #找出所有以"http"或"www"開頭且不包含當前URL的鏈結
    for link in bsObj.findAll("a", href = re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs["href"] is not None:
            if( link.attrs["href"] not in externalLinks):
                externalLinks.append(link.attrs["href"])
    return externalLinks

In [ ]:
def splitAddress(address):
    addressParts = address.replace("http://", "").split("/")
    return addressParts

In [ ]:
def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html, "lxml")
    externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0])
    if( len(externalLinks)==0):
        internalLinks = getInternalLinks(startingPage)
        return getNextExternalLink( internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

In [ ]:
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink("http://oreilly.com")
    print("隨機外鏈是:"+externalLink)
    followExternalOnly(startingSite)

In [ ]:
followExternalOnly("http://oreilly.com")

## ex2

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

In [ ]:
#收集網站上發現的所有外鏈列表
allExtLinks = set()
allIntLinks = set()

In [ ]:
def getInternalLinks(bsObj, includeUrl):
    #獲取頁面所有的內鏈列表
    internalLinks = []
    #找出所有以"/"開頭的鏈結
    for link in bsObj.findAll("a", href = re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs["href"] is not None:
            if( link.attrs["href"] not in internalLinks):
                internalLinks.append(link.attrs["href"])
    return internalLinks

In [ ]:
def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    #找出所有以"http"或"www"開頭且不包含當前URL的鏈結
    for link in bsObj.findAll("a", href = re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs["href"] is not None:
            if( link.attrs["href"] not in externalLinks):
                externalLinks.append(link.attrs["href"])
    return externalLinks

In [ ]:
def splitAddress(address):
    addressParts = address.replace("http://", "").split("/")
    return addressParts

In [ ]:
def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    bsObj = BeautifulSoup(html, "lxml")
    internalLinks = getInternalLinks(bsObj, splitAddress(siteUrl)[0])
    externalLinks = getExternalLinks(bsObj, splitAddress(siteUrl)[0])
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            print(link)    

In [ ]:
getAllExternalLinks("http://oreilly.com")

### 3.4用scrpay採集 (參考官網的scrpay教學)
#### 英文教學https://doc.scrapy.org/en/latest/intro/tutorial.html
#### 中文教學: http://scrapy-chs.readthedocs.io/zh_CN/latest/index.html

In [ ]:
#https://doc.scrapy.org/en/latest/intro/tutorial.html
